<html>
    <body>
        <center><h1>Visualize Your Data With VTK</h1></center>
        <p>The Visualization Toolkit (VTK) is an open-source, freely available software system for 3D computer
            graphics, image processing, and visualization. It consists of a C++ class library and several
            interpreted interface layers including Tcl/Tk, Java, and Python. VTK supports a wide variety of
            visualization algorithms including scalar, vector, tensor, texture, and volumetric methods, as well as
            advanced modeling techniques such as implicit modeling, polygon reduction, mesh smoothing, cutting,
            contouring, and Delaunay triangulation. VTK has an extensive information visualization framework and a
            suite of 3D interaction widgets. The toolkit supports parallel processing and integrates with various
            databases on GUI toolkits such as Qt and Tk. VTK is cross-platform and runs on Linux, Windows, Mac, and
            Unix platforms.</p>
        <p><b>Installation of VTK library: </b><code>sudo apt-get install python-vtk</code></p>
        <p>If you’re a IPython Notebook fanatic such as myself, you’re in for a let-down. VTK doesn’t come with an
            HTML output option that would allow you to directly embed the rendering results into a notebook, or any
            other web-application for that matter. Obviously, being able to interactively pan/rotate/zoom the 3D
            scene from within the notebook isn’t possible either.</p>
        <h4>Embedding VTK Renders in IPython Notebook</h4>
        <code>import vtk</code><br>
        <code>from IPython.display import Image</code><br>
        <code>def vtk_show(renderer, width=400, height=300):
        """
        Takes vtkRenderer instance and returns an IPython Image with the rendering.
        """
        renderWindow = vtk.vtkRenderWindow()
        renderWindow.SetOffScreenRendering(1)
        renderWindow.AddRenderer(renderer)
        renderWindow.SetSize(width, height)
        renderWindow.Render()
        windowToImageFilter = vtk.vtkWindowToImageFilter()
        windowToImageFilter.SetInput(renderWindow)
        windowToImageFilter.Update()
        writer = vtk.vtkPNGWriter()
        writer.SetWriteToMemory(1)
        writer.SetInputConnection(windowToImageFilter.GetOutputPort())
        writer.Write()
        data = str(buffer(writer.GetResult()))
        return Image(data) </code><br>
        <p><b>Disclaimer: </b>The code I’ll be giving in this section is not mine. I’ve come across this solution
            in the form of a ‘gist’ <a href="http://nbviewer.jupyter.org/gist/certik/5723420">here</a> , while a snippet of this code can also be found <a href="http://www.snip2code.com/Snippet/82707/Display-VTK-renders-into-IPython-Noteboo">here</a>. However, as these might be pulled down one day, I wanted to outline that code, explain how it works, and be able to refer to it in subsequent posts that will be dealing with VTK.</p>
        <p>Now let's examine the different parts of the code</p>
        <code>renderWindow = vtk.vtkRenderWindow()<br>renderWindow.SetOffScreenRendering(1)<br>renderWindow.AddRenderer(renderer)<br>renderWindow.SetSize(width, height)<br>renderWindow.Render()</code><br>
        <p>What this part does is create a new <code>vtkRenderWindow</code>, enable off-screen rendering, and adds the
            <code>vtkRenderer</code> instance to it before setting the dimensions, and performing the rendering operation.</p>
        <p><i><center><b>Note</b> that the Render method above will actually create a window separate to the
            browser where your IPython Notebook currently is. Do not mess with it or close it cause you will most
            likely get a crash.</center></i></p>
        <code>windowToImageFilter = vtk.vtkWindowToImageFilter()
windowToImageFilter.SetInput(renderWindow)
windowToImageFilter.Update()</code>
        <p>The above part creates a new <code>‘vtkWindowToImageFilter’</code> object which allows us to read the data in a
            vtkWindow and use it as input to the imaging pipeline. It then adds renderWindow to it and updates it
        </p><br>
        <code>writer = vtk.vtkPNGWriter()
writer.SetWriteToMemory(1)
writer.SetInputConnection(windowToImageFilter.GetOutputPort())
writer.Write()
data = str(buffer(writer.GetResult()))<br>    
return Image(data)</code>
        <p>Lastly, a new <code>vtkPNGWriter</code> is created allowing us to create a PNG image out of the vtkWindow rendering.
            Note that we enable <code>SetWriteToMemory</code> since we don’t just want a PNG file to be created bur rather to
            embed that image-data into the IPython Notebook. Subsequently, we establish a ‘connection’ between the
            output of the <code>‘vtkWindowToImageFilter’</code> and the <code>‘vtkPNGWriter’</code> by means of the <code>SetInputConnection</code> and
            <code>GetOutputPort</code> methods, a very typical mechanism in VTK. After using Write to create the image, we take
            that data, push in a Python buffer, and convert it to a string. Lastly, we use the Image class from the
            <code>IPython.display</code> module, imported outside the function, to convert that data to an actual image which
            can be directly embedded into the IPython Notebook.</p>
        <h4>EXAMPLE: THE RED BALL</h4>
        <p>Now lets give a quick example, to showcase the function we just presented. In the interest of
            consistency, I stole and modified this example from the original author of this function :). You can
            see more examples in that gist <a href="http://nbviewer.jupyter.org/gist/certik/5723420">here</a>.</p>
        <p>The following code creates a red sphere and uses the vtk_show function shown above to display output
            within the ipython notebook.</p>
        <code>VtkSourceSphere = vtk.vtkSphereSource()
VtkSourceSphere.SetCenter(0.0, 0.0, 0.0)
VtkSourceSphere.SetRadius(10.0)
VtkSourceSphere.SetPhiResolution(360)
VtkSourceSphere.SetThetaResolution(360)

VtkMapperSphere = vtk.vtkPolyDataMapper()
VtkMapperSphere.SetInputConnection(VtkSourceSphere.GetOutputPort())

VtkActorSphere = vtk.vtkActor()
VtkActorSphere.SetMapper(VtkMapperSphere)
VtkActorSphere.GetProperty().SetColor(1.0, 0.0, 0.0)

VtkRenderer = vtk.vtkRenderer()
VtkRenderer.SetBackground(1.0, 1.0, 1.0)
VtkRenderer.AddActor(VtkActorSphere)

vtk_show(VtkRenderer)</code>
<p>Now let us examine the code piece-by-piece:</p><br>
<code>VtkSourceSphere = vtk.vtkSphereSource()
VtkSourceSphere.SetCenter(0.0, 0.0, 0.0)
VtkSourceSphere.SetRadius(10.0)
VtkSourceSphere.SetPhiResolution(360)
VtkSourceSphere.SetThetaResolution(360)</code><br>
<p>The above code-block creates an <code>vtkSphereSource</code> object which creates a sphere represented by polygons. We use the <code>SetCenter</code> and <code>SetRadius</code> to set the coordinates of the center to the (0, 0, 0) origin of the cartesian system and set the radius of the sphere to 10.0. The <code>SetThetaResolution</code> and <code>SetPhiResolution</code> methods set the number of points in the longitude and latitude directions respectively. A low ‘resolution’ results in the sphere looking polygonal.</p><br>
<code>VtkMapperSphere = vtk.vtkPolyDataMapper()
VtkMapperSphere.SetInputConnection(VtkSourceSphere.GetOutputPort())</code><br>
<p>Consequently, we create a new <code>vtkPolyDataMapper</code> (docs <a href="https://www.vtk.org/doc/nightly/html/classvtkPolyDataMapper.html">here</a>) which maps polygonal data, such as the data representing the sphere, to graphics primitives. We then use the same <code>SetInputConnection</code> and <code>GetOutputPort</code> pipeline mechanism we saw in the previously.</p>
<p>Lastly, remember how our <code>vtk_show</code> function needs a <code>vtkRenderer</code> instance to function? Well now we have it so all we need to do is call that function as such <code>vtk_show(VtkRenderer)</code>. We could define the width and height of the rendering through a call like <code>vtk_show(VtkRenderer, width=100, height=200)</code> but we won’t bother. Upon execution you should get an image akin to the one in the figure below.</p>
<img src="https://pyscience.files.wordpress.com/2014/09/wpid-vtksphererender.png?w=584" style="width:300px;height:300px;">
<p>You can find an IPython Notebook with the code shown here under this <a href="http://nbviewer.jupyter.org/urls/bitbucket.org/somada141/pyscience/raw/master/20140904_IPythonNotebookVTK/Material/IPythonNotebookVTK.ipynb">link</a>. Note that since we’re embedding the resulting render as an IPython.display.Image object, the image should be viewable on the nbviewer which makes it great for portability.</p>
    </body>
</html>

<html>
    <body>
        <center><h1>NumPy to VTK: Converting your NumPy arrays to VTK arrays and files</h1></center>
        <p>The first way is using the <code>vtk.util.numpy_support</code> module that comes with VTK and allows you to ‘easily’ convert your data. The second way is by means of exporting your data into VTK-readable files using the <a href="https://bitbucket.org/pauloh/pyevtk">PyEVTK</a> package, a way which as you’ll see is great if you want to process and/or visualize that data in VTK-based applications.</p>
        <p><b>Using the <code>numpy_support</code> module</b></p>
        <p>So, given the popularity of Python and the fact that VTK is exposed in its near entirety to Python, the VTK folk decided to create the numpy_support module which resides under vtk.util.</p>
        <code><b>USAGE</b></code>
        <p>The functions of interest to us are <code>numpy_to_vtk</code> and <code>vtk_to_numpy</code>. Let us first inspect the docstring of the first function which can be accessed as follows, assuming you have VTK installed in your Python distro:</p><br>
        <code>from vtk.util import numpy_support
help(numpy_support.numpy_to_vtk)</code><br>
        <p>with the result being</p>
        <samp>numpy_to_vtk(num_array, deep=0, array_type=None)
    Converts a contiguous real numpy Array to a VTK array object.

    This function only works for real arrays that are contiguous.
    Complex arrays are NOT handled.  It also works for multi-component
    arrays.  However, only 1, and 2 dimensional arrays are supported.
    This function is very efficient, so large arrays should not be a
    problem.

    If the second argument is set to 1, the array is deep-copied from
    from numpy. This is not as efficient as the default behavior
    (shallow copy) and uses more memory but detaches the two arrays
    such that the numpy array can be released.

    WARNING: You must maintain a reference to the passed numpy array, if
    the numpy data is gc'd and VTK will point to garbage which will in
    the best case give you a segfault.

    Parameters
    ----------

    - num_array :  a contiguous 1D or 2D, real numpy array.</samp><br>
    <p>Upon first inspection, one might think that 3D NumPy arrays weren’t possible to convert. However, all that one needs to do is create a 1D representation of the array using ‘numpy.ndarray’ methods such as <code>flatten</code> or <code>ravel</code>. So here’s how to use this function assuming you mean to export an <code>numpy.ndarray</code> object named <code>NumPy_data</code> of type <code>float32</code> :</p><br>
    <code>NumPy_data_shape = NumPy_data.shape
VTK_data = numpy_support.numpy_to_vtk(num_array=NumPy_data.ravel(), deep=True, array_type=vtk.VTK_FLOAT)</code><br>
<p>As you can see we use <code>ravel</code> to flatten NumPy_data using the default <a href="https://en.wikipedia.org/wiki/Row-_and_column-major_order">C, or row-major, ordering</a> (especially important on 2D and 3D arrays). In addition, we specify that we do want the data to be deep-copied by setting deep=True, while we also define the data type by <code>array_type=vtk.VTK_FLOAT</code>. Note, that we keep a copy of the shape of NumPy_data which we will use later to reshape the result of <code>vtk_to_numpy</code>. Converting back is much easier and can be done as such:</p><br>
<code>NumPy_data = numpy_support.vtk_to_numpy(VTK_data)
NumPy_data = NumPy_data.reshape(NumPy_data_shape)</code><br>
<em>CAUTION: You may choose to allow for shallow-copies by setting deep=False but be warned: If for any reason, the array you pass is garbage-collected then the link will break and your nice VTK array will be useless. Should that occur, if you end up using vtk_to_numpy to back-convert, you will simply get memory trash. That is especially the case if you use flatten instead of ravel as the former always returns a ‘flattened’ copy of the array (check docs <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.flatten.html">here</a>) which is bound to get gc’ed if you use it directly in the call. ravel is slightly safer as it typically returns a new ‘view’ to the array and only copies when it has to (check docs <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.ravel.html">here</a>) but you don’t want to depend on that. Bottom line: disable deep-copying at your own risk and only if you know what you’re doing.</em>
<h4>Using the PyEVTK package</h4>
<p>What PyEVTK does, is allow you to save NumPy arrays straight to different types of VTK <a href="https://www.vtk.org/wp-content/uploads/2015/04/file-formats.pdf">XML-based file formats</a> (not the old legacy ones), without even needing to have VTK installed in your system, making NumPy the only dependency. This way, you can easily save your NumPy arrays into files that can be visualized and processed with any of the flagship VTK applications such as ParaView, VisIt, Mayavi, while obviously you can easily load said files with VTK and use all the goodies the toolkit offers.</p><br>
<b><code>Usage</code></b>
<p>I’m gonna give a quick example here to show you how to save a NumPy array as a rectilinear-grid file with a <code>.vtr</code> extension. Code for creating a <a href="https://en.wikipedia.org/wiki/Julia_set">Julia set</a> is modified</p>
<p>Here, however, I’m only going to show the PyEVTK part of the code while you can see the full thing under this <a href="http://nbviewer.jupyter.org/urls/bitbucket.org/somada141/pyscience/raw/master/20140905_NumPyToVTK/Material/DemoPyEVTK.ipynb">IPython Notebook</a>. The result of the Julia code is the following pretty 2D array which was visualized with <a href="https://plot.ly/python/heatmaps/">Plotly’s Heatmap method</a></p>
<img src="https://pyscience.files.wordpress.com/2014/09/wpid-julia.png?w=584" style="width:300px;height:300px;">
<p>So, here’s the PyEVTK part of the notebook:</p>
<code>from pyevtk.hl import gridToVTK

noSlices = 5
juliaStacked = numpy.dstack([julia]*noSlices)

x = numpy.arange(0, w+1)
y = numpy.arange(0, h+1)
z = numpy.arange(0, noSlices+1)

gridToVTK("./julia", x, y, z, cellData = {'julia': juliaStacked})</code>
<p>The first important part of this code is from <code>pyevtk.hl import gridToVTK</code> which uses the high-level functions of the <code>pyevtk</code> package and which reside under the <code>pyevtk.hl</code> module. For the most-part, this module has all you need but the package also provides a <code>pyevtk.vtk</code> module for low-level calls. As we want to export data residing on a rectilinear grid we use the <code>gridToVTK</code> method.
</p>
<p>The next 5 lines aren’t important but what we do is use the <code>numpy.dstack</code> function to stack 5 copies of the <code>2D julia</code> array and create a 3D array to export. Note the <a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html">numpy.arange</a> calls through which we calculate ‘fake’ axes for the array to be exported. An important point to make is that since we’re defining a grid we need axes with N+1 coordinates which define the grid edges (as the data resides in the centers of that grid). If you want to export point-data, use the <code>pyevtk.hl.pointsToVTK</code> function.</p>
<p>As you guessed the magic happens with this one line</p><br>
<code>gridToVTK("./julia", x, y, z, cellData = {'julia': juliaStacked})</code><br>
<p>As you can see, we pass the path to the file <b>without an extension</b> which will be automatically defined by PyEVTK depending on which function you used in order to create the appropriate VTK file. Subsequently, we pass the three coordinate axes, and provide a Python dictionary with a name for the array and the array itself. Doesn’t get easier than that :). The result of the above is <code>.vtr</code> file which you can find <a href="https://bitbucket.org/somada141/pyscience/raw/master/20140905_NumPyToVTK/Material/julia.vtr">here</a> while the IPython Notebook of the entire code can be found <a href="http://nbviewer.jupyter.org/urls/bitbucket.org/somada141/pyscience/raw/master/20140905_NumPyToVTK/Material/DemoPyEVTK.ipynb">here</a>.

</p>
    </body>
</html>